In [32]:
import os
import numpy as np
import pandas as pd
import math
import matplotlib.patches as patches
import matplotlib.pylab as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.cross_validation import cross_val_score
from sklearn import tree
from sklearn import metrics
from sklearn import datasets
from IPython.display import Image
%matplotlib inline
from pandas import DataFrame, read_sql
import pandas as pd
import sqlite3
from sklearn.cluster import KMeans

In [7]:
con = sqlite3.connect("gov_zip_censys.db")

cert_sql = '''SELECT *
                FROM cert_data
                '''

df_cert = pd.read_sql(cert_sql, con)
len(df_cert)

13615

In [8]:
def map_values_to_nums(dataframe, df_column):
    value_num_map = {}
    count = 1
    for x in df_column:
        if x not in value_num_map:
            value_num_map[x] = count
            count += 1
    value_num_list = []
    for x in df_column:
        value_num_list.append(value_num_map[x])
    dataframe[df_column.name+"_nums"] = value_num_list
    return dataframe, value_num_map

def map_values_to_nums_all(dataframe):
    dataframe, signing_algo_dict = map_values_to_nums(dataframe,dataframe.signing_algorithm)
    dataframe, key_algo_dict = map_values_to_nums(dataframe,dataframe.key_algorithm)
    dataframe, subject_cn_dict = map_values_to_nums(dataframe, dataframe.subject_cn)
    dataframe, subject_c_dict = map_values_to_nums(dataframe,dataframe.subject_c)
    dataframe, subject_o_dict = map_values_to_nums(dataframe, dataframe.subject_o)
    dataframe, issuer_cn_dict = map_values_to_nums(dataframe, dataframe.issuer_cn)
    dataframe, issuer_c_dict = map_values_to_nums(dataframe, dataframe.issuer_c)
    dataframe, issuer_o_dict = map_values_to_nums(dataframe, dataframe.issuer_o)
    result = {}
    result["signing_algo_dict"] = signing_algo_dict 
    result["key_algo_dict"] = key_algo_dict
    result["subject_cn_dict"] = subject_cn_dict
    result["subject_c_dict"] = subject_c_dict
    result["subject_o_dict"] = subject_o_dict
    result["issuer_cn_dict"] = issuer_cn_dict
    result["issuer_c_dict"] = issuer_c_dict
    result["issuer_o_dict"] = issuer_o_dict
    return result

In [9]:
result = map_values_to_nums_all(df_cert)

In [10]:
result["signing_algo_dict"],result["key_algo_dict"]

({u'ECDSAWitHSHA384': 8,
  u'ECDSAWithSHA256': 3,
  u'ECDSAWithSHA512': 9,
  u'MD5WithRSA': 4,
  u'SHA1WithRSA': 2,
  u'SHA256WithRSA': 1,
  u'SHA384WithRSA': 7,
  u'SHA512WithRSA': 6,
  u'unknown_algorithm': 5},
 {u'DSA': 3, u'ECDSA': 2, u'RSA': 1})

In [11]:
def create_columns(result):
    columns = []
    for x in result["signing_algo_dict"]:
        columns.append("signing_algo."+x)
    for x in result["key_algo_dict"]:
        columns.append("key_algo."+x)
    return columns

In [14]:
t_columns = create_columns(result)
t_columns

[u'signing_algo.SHA256WithRSA',
 u'signing_algo.MD5WithRSA',
 u'signing_algo.ECDSAWithSHA256',
 u'signing_algo.SHA384WithRSA',
 u'signing_algo.unknown_algorithm',
 u'signing_algo.SHA1WithRSA',
 u'signing_algo.SHA512WithRSA',
 u'signing_algo.ECDSAWitHSHA384',
 u'signing_algo.ECDSAWithSHA512',
 u'key_algo.DSA',
 u'key_algo.RSA',
 u'key_algo.ECDSA']

In [16]:
def create_data(dataframe, columns):
    n_array = []
    for x in range(len(dataframe)):
        temp = [0]*12
        for y in range(0,9):
            if dataframe.signing_algorithm[x] == columns[y].split('.')[1]:
                temp[y] = 1
        for y in range(9,12):
            if dataframe.key_algorithm[x] == columns[y].split('.')[1]:
                temp[y] = 1
        n_array.append(temp)
    return n_array

In [17]:
t_data = create_data(df_cert, t_columns)

In [29]:
data = pd.DataFrame(data=t_data, columns=t_columns)
data

,signing_algo.SHA256WithRSA,signing_algo.MD5WithRSA,signing_algo.ECDSAWithSHA256,signing_algo.SHA384WithRSA,signing_algo.unknown_algorithm,signing_algo.SHA1WithRSA,signing_algo.SHA512WithRSA,signing_algo.ECDSAWitHSHA384,signing_algo.ECDSAWithSHA512,key_algo.DSA,key_algo.RSA,key_algo.ECDSA
0,1,0,0,0,0,0,0,0,0,0,1,0
1,1,0,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,1,0,0,0,0,1,0
3,1,0,0,0,0,0,0,0,0,0,1,0
4,1,0,0,0,0,0,0,0,0,0,1,0
5,1,0,0,0,0,0,0,0,0,0,1,0
6,1,0,0,0,0,0,0,0,0,0,1,0
7,1,0,0,0,0,0,0,0,0,0,1,0
8,1,0,0,0,0,0,0,0,0,0,1,0
9,1,0,0,0,0,0,0,0,0,0,1,0


In [30]:
k_clusters = 2

# Fit and predict clusters
model = KMeans(k_clusters)
model.fit(data)
clusters = model.predict(data)

# Do some messy stuff to print a nice table of clusters
cluster_listing = {}
for cluster in range(k_clusters):
    cluster_listing['Cluster ' + str(cluster)] = [''] * 100000
    where_in_cluster = np.where(clusters == cluster)[0]
    cluster_listing['Cluster ' + str(cluster)][0:len(where_in_cluster)] = data.index[where_in_cluster]

# Print clusters
pd.DataFrame(cluster_listing).loc[0:np.max(np.bincount(clusters)) - 1,:]

,Cluster 0,Cluster 1
0,0,2
1,1,34
2,3,262
3,4,306
4,5,320
5,6,400
6,7,521
7,8,522
8,9,557
9,10,602


In [31]:
data.insert(0,'cluster',clusters)
center = data.groupby(['cluster']).mean()
center
data

,cluster,signing_algo.SHA256WithRSA,signing_algo.MD5WithRSA,signing_algo.ECDSAWithSHA256,signing_algo.SHA384WithRSA,signing_algo.unknown_algorithm,signing_algo.SHA1WithRSA,signing_algo.SHA512WithRSA,signing_algo.ECDSAWitHSHA384,signing_algo.ECDSAWithSHA512,key_algo.DSA,key_algo.RSA,key_algo.ECDSA
0,0,1,0,0,0,0,0,0,0,0,0,1,0
1,0,1,0,0,0,0,0,0,0,0,0,1,0
2,1,0,0,0,0,0,1,0,0,0,0,1,0
3,0,1,0,0,0,0,0,0,0,0,0,1,0
4,0,1,0,0,0,0,0,0,0,0,0,1,0
5,0,1,0,0,0,0,0,0,0,0,0,1,0
6,0,1,0,0,0,0,0,0,0,0,0,1,0
7,0,1,0,0,0,0,0,0,0,0,0,1,0
8,0,1,0,0,0,0,0,0,0,0,0,1,0
9,0,1,0,0,0,0,0,0,0,0,0,1,0
